In [ ]:
import os
import boto3
import mlflow
import json
import shutil

## Verify AWS Role Assignment

Only the prod-user should have permissions to read directly from the S3 bucket for DOMINO_BLOBS
In this step verify that the role is being assumed.

In [ ]:
os.environ['AWS_WEB_IDENTITY_TOKEN_FILE']='/var/run/secrets/eks.amazonaws.com/serviceaccount/token'
sts_client = boto3.client('sts')
identity = sts_client.get_caller_identity()
print(identity)

In [ ]:
REGISTERED_MODEL_NAME="TinyLlama-MODEL"
REGISTERED_MODEL_VERSION="8"


In [ ]:
def get_all_model_version(registered_model_name):
    client = mlflow.tracking.MlflowClient()
    # List all versions for the model
    model_versions = client.search_model_versions(f"name='{registered_model_name}'")

    for version in model_versions:
        print(f"Version: {version.version}, Status: {version.status}, Stage: {version.current_stage}")

In [ ]:
def get_parent_run_id_for_model_version(registered_model_name,registered_model_version):
    client = mlflow.tracking.MlflowClient()
    model_version_info = client.get_model_version(name=registered_model_name, version=registered_model_version)

    # Extract the run ID associated with this version
    run_id = model_version_info.run_id

    # Build artifact path
    artifact_relative_path = "artifacts/model_context.json"

    # Set where you want to download
    download_dir = "/tmp"
    os.makedirs(download_dir, exist_ok=True)
    local_path = client.download_artifacts(run_id=run_id, path=artifact_relative_path, dst_path=download_dir)

    # Load JSON from a file
    with open(local_path, 'r') as f:
        data = json.load(f)

    # Extract the run_id attribute
    run_id = data.get('run_id')

    return run_id

In [ ]:
def download_s3_folder(bucket_name, run_id, prod_ds_folder):
    s3 = boto3.client('s3')

    # Full path where we want to download
    target_local_folder = os.path.join(prod_ds_folder, run_id)

    # Check if parent folder exists and is writable
    if not os.path.isdir(prod_ds_folder):
        raise Exception(f"Base directory {prod_ds_folder} does not exist. Cannot proceed.")

    if not os.access(prod_ds_folder, os.W_OK):
        raise Exception(f"Base directory {prod_ds_folder} is not writable. Cannot proceed.")

    # If run_id folder already exists, delete it
    if os.path.exists(target_local_folder):
        shutil.rmtree(target_local_folder)

    # Create clean run_id folder
    os.makedirs(target_local_folder, exist_ok=True)

    # List all objects under mlflow/{run_id}/artifacts/model/
    prefix = f"mlflow/{run_id}/artifacts/model/"
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    for page in pages:
        for obj in page.get('Contents', []):
            key = obj['Key']
            if key.endswith('/'):
                continue  # Skip "folder" markers

            # Download each file
            relative_path = key[len(prefix):]
            local_file_path = os.path.join(target_local_folder, relative_path)

            os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
            print(f"Downloading {key} to {local_file_path}")
            s3.download_file(bucket_name, key, local_file_path)

## Fetch Model Artifacts from Experiment Manager

Select you model name and model version and download the model artifacts from the `mlflow_run_id`
associated with this model name and model version

Note that we enable IRSA for this workspace and this prod deployer user has permissions to:

1. List/Read Domino `BLOBS` bucket
2. Fetch the `parent_run_id` from the `registered_model_name/registered_model_version`. The function
   is `get_parent_run_id_for_model_version`
   - Fetch the `mlflow_run_id` associated with this registered model version
   - The artifact `artifacts/model_context.json` associated with this `mlflow_run_id` 
   - The `parent_run_id` is contained in this json in this attribute `run_id`
3. Now use the function `download_s3_folder` to download artifacts from the bucket location
   `mlflow/{run_id}/artifacts/model/` into the location `{PROD_DATASET}/llm-models/{parent_run_id}/

In [ ]:
#get_all_model_version(registered_model_name)
run_id = get_parent_run_id_for_model_version(REGISTERED_MODEL_NAME,REGISTERED_MODEL_VERSION)
print(run_id)

BUCKET_NAME=os.environ['MLFLOW_S3_BUCKET']
LLM_ARTIFACTS_RUN_ID=get_parent_run_id_for_model_version(REGISTERED_MODEL_NAME,REGISTERED_MODEL_VERSION)
PROD_DS_FOLDER="/domino/datasets/domino-models-prod/llm-models"

print(f"BUCKET_NAME={BUCKET_NAME}, LLM_ARTIFACTS_RUN_ID={LLM_ARTIFACTS_RUN_ID}")
download_s3_folder(BUCKET_NAME,run_id,PROD_DS_FOLDER)